In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os

2025-07-29 11:37:14.163575: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = "plantData/Image Data base/Image Data base"

In [3]:
x_train = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split = 0.2,
        subset="training",
        seed=123,
        image_size=(224, 224),
        batch_size=64
    )

Found 199665 files belonging to 58 classes.
Using 159732 files for training.


In [4]:
x_test = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split = 0.2,
        subset="validation",
        seed=123,
        image_size=(224,224),
        batch_size=64
    )

Found 199665 files belonging to 58 classes.
Using 39933 files for validation.


In [5]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
predictions = Dense(len(x_train.class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
history = model.fit(
    x_train,
    validation_data=x_test,
    epochs=5
)

Epoch 1/5
2496/2496 ━━━━━━━━━━━━━━━━━━━━ 6437s 3s/step - accuracy: 0.8552 - loss: 0.6200 - val_accuracy: 0.9727 - val_loss: 0.1057
Epoch 2/5
2496/2496 ━━━━━━━━━━━━━━━━━━━━ 6385s 3s/step - accuracy: 0.9666 - loss: 0.1163 - val_accuracy: 0.9801 - val_loss: 0.0710
Epoch 3/5
2496/2496 ━━━━━━━━━━━━━━━━━━━━ 6363s 3s/step - accuracy: 0.9738 - loss: 0.0868 - val_accuracy: 0.9832 - val_loss: 0.0585
Epoch 4/5
2496/2496 ━━━━━━━━━━━━━━━━━━━━ 6405s 3s/step - accuracy: 0.9766 - loss: 0.0741 - val_accuracy: 0.9854 - val_loss: 0.0506
Epoch 5/5
2496/2496 ━━━━━━━━━━━━━━━━━━━━ 6390s 3s/step - accuracy: 0.9786 - loss: 0.0672 - val_accuracy: 0.9854 - val_loss: 0.0474


In [ ]:
model.save("efficientnet_custom_model_1.keras")

In [ ]:
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5),  # Lower LR when fine-tuning
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_finetune = model.fit(
    x_train,
    validation_data=x_test,
    epochs=5
)

In [ ]:
model.save("efficientnet_custom_model.keras")